---
title: "Poisson Regression Examples"
author: "Your Name"
date: today
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---


## Blueprinty Case Study

### Introduction

Blueprinty is a small firm that makes software for developing blueprints specifically for submitting patent applications to the US patent office. Their marketing team would like to make the claim that patent applicants using Blueprinty's software are more successful in getting their patent applications approved. Ideal data to study such an effect might include the success rate of patent applications before using Blueprinty's software and after using it. Unfortunately, such data is not available. 

However, Blueprinty has collected data on 1,500 mature (non-startup) engineering firms. The data include each firm's number of patents awarded over the last 5 years, regional location, age since incorporation, and whether or not the firm uses Blueprinty's software. The marketing team would like to use this data to make the claim that firms using Blueprinty's software are more successful in getting their patent applications approved.


### Data


In [ ]:
import pandas as pd
import seaborn as sns

# Load the Blueprinty data
data = pd.read_csv('/home/jovyan/Desktop/marketingwebsite/blueprinty.csv')

# Display the first few rows of the dataset
data.head()

_todo: Compare histograms and means of number of patents by customer status. What do you observe?_


In [ ]:
import matplotlib.pyplot as plt

# Separate data into customers and non-customers
customers = data[data['iscustomer'] == 1]
non_customers = data[data['iscustomer'] == 0]

# Plot histograms
plt.hist(customers['patents'], bins=20, alpha=0.5, label='Customers')
plt.hist(non_customers['patents'], bins=20, alpha=0.5, label='Non-Customers')
plt.xlabel('Number of Patents')
plt.ylabel('Frequency')
plt.legend()
plt.title('Histogram of Number of Patents by Customer Status')
plt.show()

# Calculate and print means
mean_customers = customers['patents'].mean()
mean_non_customers = non_customers['patents'].mean()
print(f"Mean number of patents for customers: {mean_customers}")
print(f"Mean number of patents for non-customers: {mean_non_customers}")

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.

_todo: Compare regions and ages by customer status. What do you observe?_


In [ ]:
# Compare regions by customer status
region_counts_customers = customers['region'].value_counts(normalize=True)
region_counts_non_customers = non_customers['region'].value_counts(normalize=True)

print("Region distribution for customers:")
print(region_counts_customers)
print("\nRegion distribution for non-customers:")
print(region_counts_non_customers)

# Compare ages by customer status
mean_age_customers = customers['age'].mean()
mean_age_non_customers = non_customers['age'].mean()

print(f"\nMean age for customers: {mean_age_customers}")
print(f"Mean age for non-customers: {mean_age_non_customers}")

### Estimation of Simple Poisson Model

Since our outcome variable of interest can only be small integer values per a set unit of time, we can use a Poisson density to model the number of patents awarded to each engineering firm over the last 5 years. We start by estimating a simple Poisson model via Maximum Likelihood.

_todo: Write down mathematically the likelihood for_ $Y \sim \text{Poisson}(\lambda)$. Note that $f(Y|\lambda) = e^{-\lambda}\lambda^Y/Y!$.


In [ ]:
# Define the Poisson likelihood function
import numpy as np

def poisson_likelihood(lambda_, Y):
   """
   Compute the Poisson likelihood for given lambda and observed Y.
   """
   likelihood = np.exp(-lambda_) * (lambda_ ** Y) / np.math.factorial(Y)
   return likelihood

# Example usage
Y_observed = 5  # Example observed number of patents
lambda_example = 3.0  # Example lambda value
likelihood_value = poisson_likelihood(lambda_example, Y_observed)
print(f"Poisson likelihood for Y={Y_observed} and lambda={lambda_example}: {likelihood_value}")

_todo: Code the likelihood (or log-likelihood) function for the Poisson model. This is a function of lambda and Y. For example:_

```
poisson_loglikelihood <- function(lambda, Y){
   ...
}
```


In [ ]:
def poisson_loglikelihood(lambda_, Y):
   """
   Compute the Poisson log-likelihood for given lambda and observed Y.
   """
   from scipy.special import factorial
   log_likelihood = np.sum(Y * np.log(lambda_) - lambda_ - np.log(factorial(Y)))
   return log_likelihood

_todo: Use your function to plot lambda on the horizontal axis and the likelihood (or log-likelihood) on the vertical axis for a range of lambdas (use the observed number of patents as the input for Y)._


In [ ]:
# Define a range of lambda values
lambda_values = np.linspace(0.1, 10, 100)

# Convert Y_sample to a NumPy array
Y_sample = np.array([5, 3, 4, 6, 2])  # Example observed data

# Compute the log-likelihood for each lambda
log_likelihoods = [poisson_loglikelihood(l, Y_sample) for l in lambda_values]

# Plot the log-likelihood
plt.plot(lambda_values, log_likelihoods, label='Log-Likelihood')
plt.xlabel('Lambda')
plt.ylabel('Log-Likelihood')
plt.title('Log-Likelihood vs Lambda')
plt.axvline(x=np.mean(Y_sample), color='red', linestyle='--', label='MLE (Mean of Y)')
plt.legend()
plt.show()

_todo: If you're feeling mathematical, take the first derivative of your likelihood or log-likelihood, set it equal to zero and solve for lambda. You will find lambda_mle is Ybar, which "feels right" because the mean of a Poisson distribution is lambda._


In [ ]:
# Derive the first derivative of the log-likelihood function
# Log-likelihood: l(lambda) = Y * log(lambda) - lambda - log(Y!)
# First derivative: dl/dlambda = Y / lambda - 1

# Solve for lambda when dl/dlambda = 0
# Y / lambda - 1 = 0 => lambda = Y

# For a sample of observations, the MLE for lambda is the sample mean (Ybar)
def compute_lambda_mle(Y):
   """
   Compute the MLE for lambda (mean of Y).
   """
   return np.mean(Y)

# Example usage
Y_sample = [5, 3, 4, 6, 2]  # Example observed data
lambda_mle = compute_lambda_mle(Y_sample)
print(f"MLE for lambda (mean of Y): {lambda_mle}")

_todo: Find the MLE by optimizing your likelihood function with optim() in R or sp.optimize() in Python._


In [ ]:
from scipy.optimize import minimize

# Define the negative log-likelihood function for optimization
def negative_log_likelihood(lambda_, Y):
   """
   Compute the negative log-likelihood for Poisson.
   from scipy.special import factorial
   return -np.sum(Y * np.log(lambda_) - lambda_ - np.log(factorial(Y)))
   return -np.sum(Y * np.log(lambda_) - lambda_ - np.log(np.array([np.math.factorial(y) for y in Y])))

# Observed data
Y_sample = [5, 3, 4, 6, 2]  # Example observed data

# Initial guess for lambda
initial_guess = np.mean(Y_sample)

# Optimize the negative log-likelihood
result = minimize(lambda l: negative_log_likelihood(l, Y_sample), x0=initial_guess, bounds=[(0.1, None)])

# Extract the MLE for lambda
lambda_mle = result.x[0]
print(f"MLE for lambda: {lambda_mle}")

### Estimation of Poisson Regression Model

Next, we extend our simple Poisson model to a Poisson Regression Model such that $Y_i = \text{Poisson}(\lambda_i)$ where $\lambda_i = \exp(X_i'\beta)$. The interpretation is that the success rate of patent awards is not constant across all firms ($\lambda$) but rather is a function of firm characteristics $X_i$. Specifically, we will use the covariates age, age squared, region, and whether the firm is a customer of Blueprinty.

_todo: Update your likelihood or log-likelihood function with an additional argument to take in a covariate matrix X. Also change the parameter of the model from lambda to the beta vector. In this model, lambda must be a positive number, so we choose the inverse link function g_inv() to be exp() so that_ $\lambda_i = e^{X_i'\beta}$. _For example:_

```
poisson_regression_likelihood <- function(beta, Y, X){
   ...
}
```


In [ ]:
def poisson_regression_loglikelihood(beta, Y, X):
   """
   Compute the Poisson regression log-likelihood for given beta, observed Y, and covariate matrix X.
   """
   # Compute lambda_i = exp(X_i' * beta)
   lambda_ = np.exp(np.dot(X, beta))
   
   # Compute the log-likelihood
   log_likelihood = np.sum(Y * np.log(lambda_) - lambda_ - np.log(np.array([np.math.factorial(y) for y in Y])))
   return log_likelihood

_todo: Use your function along with R's optim() or Python's sp.optimize() to find the MLE vector and the Hessian of the Poisson model with covariates. Specifically, the first column of X should be all 1's to enable a constant term in the model, and the subsequent columns should be age, age squared, binary variables for all but one of the regions, and the binary customer variable. Use the Hessian to find standard errors of the beta parameter estimates and present a table of coefficients and standard errors._

_todo: Check your results using R's glm() function or Python sm.GLM() function._

_todo: Interpret the results._ 

_todo: What do you conclude about the effect of Blueprinty's software on patent success? Because the beta coefficients are not directly interpretable, it may help to create two fake datasets: X_0 and X_1 where X_0 is the X data but with iscustomer=0 for every observation and X_1 is the X data but with iscustomer=1 for every observation. Then, use X_0 and your fitted model to get the vector of predicted number of patents (y_pred_0) for every firm in the dataset, and use X_1 to get Y_pred_1 for every firm. Then subtract y_pred_1 minus y_pred_0 and take the average of that vector of differences._




## AirBnB Case Study

### Introduction

AirBnB is a popular platform for booking short-term rentals. In March 2017, students Annika Awad, Evan Lebo, and Anna Linden scraped of 40,000 Airbnb listings from New York City.  The data include the following variables:

:::: {.callout-note collapse="true"}
### Variable Definitions

    - `id` = unique ID number for each unit
    - `last_scraped` = date when information scraped
    - `host_since` = date when host first listed the unit on Airbnb
    - `days` = `last_scraped` - `host_since` = number of days the unit has been listed
    - `room_type` = Entire home/apt., Private room, or Shared room
    - `bathrooms` = number of bathrooms
    - `bedrooms` = number of bedrooms
    - `price` = price per night (dollars)
    - `number_of_reviews` = number of reviews for the unit on Airbnb
    - `review_scores_cleanliness` = a cleanliness score from reviews (1-10)
    - `review_scores_location` = a "quality of location" score from reviews (1-10)
    - `review_scores_value` = a "quality of value" score from reviews (1-10)
    - `instant_bookable` = "t" if instantly bookable, "f" if not

::::


_todo: Assume the number of reviews is a good proxy for the number of bookings. Perform some exploratory data analysis to get a feel for the data, handle or drop observations with missing values on relevant variables, build one or more models (e.g., a poisson regression model for the number of bookings as proxied by the number of reviews), and interpret model coefficients to describe variation in the number of reviews as a function of the variables provided._


In [ ]:
# Load the Blueprinty data
airbnb = pd.read_csv('/home/jovyan/Desktop/marketingwebsite/airbnb.csv')

# Display the first few rows of the dataset
airbnb.head()

In [ ]:
print(airbnb.shape)
print(airbnb.info())

In [ ]:
# Show the number of missing values in each column
missing_values = airbnb.isnull().sum()
print("Number of missing values in each column:")
print(missing_values)

In [ ]:
airbnb.dropna(subset=['review_scores_cleanliness', 'review_scores_location', 'review_scores_value'], inplace=True)

In [ ]:
airbnb.describe(include='object')

In [ ]:
airbnb['last_scraped'] = pd.to_datetime(airbnb['last_scraped'])
airbnb['host_since'] = pd.to_datetime(airbnb['host_since'])

airbnb['days'] = (airbnb['last_scraped'] - airbnb['host_since']).dt.days

Correlation Matrix for Numerical Variables

In [ ]:
numeric_cols = ['price', 'bathrooms', 'bedrooms', 'number_of_reviews', 
                'review_scores_cleanliness', 'review_scores_location', 
                'review_scores_value', 'days']
sns.heatmap(airbnb[numeric_cols].corr(), annot=True)

Price Distribution

In [ ]:
sns.histplot(airbnb['price'], bins=50)

Price vs. Room Type

In [ ]:
sns.boxplot(x='room_type', y='price', data=airbnb)